<a href="https://colab.research.google.com/github/asadprl/brain-tumor-deep-learning/blob/main/CS619_Prototype_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
 # **Brain Tumor Detection through Transfer Learning**
---
## **Protype Phase Assignment**

###**Problem Statement:**



The prototype involves the following steps:

1.	Collect a diverse dataset of medical images, including MRI.
2.	Preprocess the dataset, ensuring uniformity in terms of image size, format, and quality.
3.	Select pre-trained deep learning models suitable for transfer learning, such as including Visual Geometry Group 16 (VGG16), InceptionV3, VGG19, ResNet50, InceptionResNetV2, and Xception. You should select two well-established pre-trained models.

###**Solution:**

####**1. Dataset**

For this assignment I have used the MRI images dataset found on Kaggle from [this link](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset).
This dataset is a combination of following datasets:
>1. [Figshare](https://figshare.com/articles/dataset/brain_tumor_dataset/1512427)
2. [Sartaj](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri)
3. [BR35H](https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection?select=no)

The Dataset contains total 7023 MRI images of human brain which are divided into 4 classes:
>1. Glioma
2. Meningioma
3. Pituitory
4. Normal

#####**1.1 Importing Necessary Libraries and Modules**


In [ ]:
import os
import gdown
import zipfile
from ipywidgets import interact
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19

#####**1.2 Configuration Settings**


In [ ]:
# path to different directories
dataset_path = '/content/dataset'
train_dir = os.path.join(dataset_path, 'Training')
test_dir = os.path.join(dataset_path, 'Testing')

image_size = (224, 224)
batch_size = 32

#####**1.3 Downloading Dataset**
I have stored the dataset in my gdrive so downloading the dataset to current environment. It will be saved in /content directory

In [ ]:
dataset_url = 'https://drive.google.com/uc?id=1Rt-nwEiY9UjWcZRrRRsQS7EN129DwFwV'
dataset_file = 'brain_tumor_dataset.zip'
gdown.download(url=dataset_url, output=dataset_file)

Downloading...
From (original): https://drive.google.com/uc?id=1Rt-nwEiY9UjWcZRrRRsQS7EN129DwFwV
From (redirected): https://drive.google.com/uc?id=1Rt-nwEiY9UjWcZRrRRsQS7EN129DwFwV&confirm=t&uuid=1671cad6-591a-4936-b1c6-eaae6e6e9cb1
To: /content/brain_tumor_dataset.zip
100%|██████████| 156M/156M [00:00<00:00, 208MB/s]


'brain_tumor_dataset.zip'

#####**1.4 Extracting Dataset**
Dataset is in zip format so extracting it to dataset directory. "/content/dataset"

In [ ]:
zip_ref = zipfile.ZipFile(dataset_file, 'r')
zip_ref.extractall(dataset_path)

The dataset is already divided into two subdirectories. Testing and Training. So we don't need to seperate the dataset into two subsets it is already divided and also dataset structure is compatible with keras' `flow_from_directory` method. It mean each Training and Testing subsets has subdirectories for each class like:
- Testing
 - glioma
 - meningioma
 - notumor
 - pituitary
- Training
 - glioma
 - meningioma
 - notumor
 - pituitary

####**2. Preprocessing**

Training Data will be divided into two part, for training and validation while Testing Data will be seperate that will not be used during training. 80% data in Training directory will be used for training and remaining 20% in this directory will be used for validation purpose. Data will be loaded using keras' ImageDataGenerator. All the images will be rescaled to target size. This will ensure the uniformity in terms of image size and format. As all the images will be rescaled to required size and format realtime during the training. During training, training and validation data will be augmented realtime using datagenerator while the testing data will be remain same. Only rescaling will be applied to this data.

In [ ]:
# ImageDataGenerator for Training and Validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                validation_split=0.2)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=image_size,
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    subset='training')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=image_size,
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    subset='validation')

# ImageDataGenerator for Testing
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=image_size,
                                                  class_mode='categorical',
                                                  batch_size=batch_size)

Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
# some sample images
images, labels =  val_generator.next()

@interact(idx=(0,batch_size-1))
def display_image(idx):
  plt.imshow(images[idx])
  plt.show()

interactive(children=(IntSlider(value=15, description='idx', max=31), Output()), _dom_classes=('widget-interac…

####**3. Transfer Learning Models**

In [ ]:
class PretrainedModel:
    def __init__(self):
        self.base_model = None
        self.model = None

    def load_base_model(self, model):
        self.base_model = model(
            weights = 'imagenet',
            include_top = False,
            input_shape = (image_size + (3,))
        )

    def freeze_layers(self):
        for layer in self.base_model.layers:
            layer.trainable = False
        self.base_model.trainable = False

    def create_final_model(self):
        pass

    def summary(self):
        return self.model.summary()


#####**3.1 InceptionV3**

In [ ]:
class InceptionV3Model(PretrainedModel):
    def __init__(self):
        super().__init__()

    def load_base_model(self):
        super().load_base_model(InceptionV3)

    def create_final_model(self):
        self.freeze_layers()

        last_output = self.base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
        x = tf.keras.layers.Dense(128, activation='relu', name='new_dense_1')(x)
        x = tf.keras.layers.Dense(4, activation='softmax',name='new_output')(x)

        self.model = tf.keras.models.Model(inputs=self.base_model.input, outputs=x)

        self.model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])



In [ ]:
inception_model = InceptionV3Model()
inception_model.load_base_model()
inception_model.create_final_model()
inception_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_658 (Conv2D)         (None, 111, 111, 32)         864       ['input_8[0][0]']             
                                                                                                  
 batch_normalization_658 (B  (None, 111, 111, 32)         96        ['conv2d_658[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_658 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_65

#####**3.2 VGG19**

In [ ]:
class VGG19Model(PretrainedModel):
    def __init__(self):
        super().__init__()

    def load_base_model(self):
        super().load_base_model(VGG19)

    def create_final_model(self):
        self.freeze_layers()

        last_output = self.base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
        x = tf.keras.layers.Dense(128, activation='relu', name='new_dense_1')(x)
        x = tf.keras.layers.Dense(4, activation='softmax',name='new_output')(x)

        self.model = tf.keras.models.Model(inputs=self.base_model.input, outputs=x)

        self.model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])



In [ ]:
vgg_model = VGG19Model()
vgg_model.load_base_model()
vgg_model.create_final_model()
vgg_model.summary()

80134624/80134624 [==============================] - 3s 0us/step
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            